In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt
import calmap
import plotly.express as px
import plotly.graph_objects as go

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


import warnings
warnings.filterwarnings('ignore')

In [2]:
path = r'C:/Users/frede/Documents/GitHub/BC5_Demand_Forecast/data/MindOverData_RetailChallenge.csv'
pathPP = r'C:/Users/frede/Documents/GitHub/BC5_Demand_Forecast/data/values_units.csv'

In [ ]:
# 20 M
data =  pd.read_csv(path, nrows=1999998)

In [ ]:
#PreProcessing - removing excessive letters from the variables

data['ProductFamily_ID'] = data['ProductFamily_ID'].str.split('_').str[1]
data['ProductCategory_ID'] = data['ProductCategory_ID'].str.split('_').str[1]
data['ProductBrand_ID'] = data['ProductBrand_ID'].str.split('_').str[1]
data['ProductName_ID'] = data['ProductName_ID'].str.split('_').str[1]
data['ProductPackSKU_ID'] = data['ProductPackSKU_ID'].str.split('_').str[1]
data['Point-of-Sale_ID'] = data['Point-of-Sale_ID'].str.split('_').str[1]

data['Measures'] = data['Measures'].str.split(' ').str[1]

data.head(20)

In [ ]:
data.info()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
#data = data.sort_values(by='Date')

In [ ]:
# Creating a variable with the day of the week of each purchase

data['Day of the Week'] = data['Date'].dt.dayofweek

In [ ]:
# Creating a variable with the correspondent quarter

data['Quarter'] = data['Date'].dt.quarter

In [ ]:
# Splitting the dataframe based on Units/ Values

data_units = data[data['Measures'] == 'units']
data_values = data[data['Measures'] == 'values']

In [ ]:
data_values = data_values.assign(Units=data_units['Value'].values)
data_values = data_values.drop(columns='Measures')

In [ ]:
#data_values.to_csv('values_units.csv.gz', compression='gzip')

##### Loading the preprocessed dataset

In [ ]:
dataPP =  pd.read_csv(pathPP,  nrows=100000,dtype={'ProductFamily_ID': str, 'Point-of-Sale_ID': str})

In [ ]:
#dataPP[["ProductFamily_ID", "Point-of-Sale_ID"]] = dataPP[["ProductFamily_ID", "Point-of-Sale_ID"]].astype(str) 

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="Day of the Week", data=dataPP, color='darkblue')
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of the week', fontsize=12)
plt.title("Frequency of purchase by day of the week", fontsize=15)
plt.show()

In [ ]:
cnt = dataPP['ProductName_ID'].value_counts()[:10]
plt.figure(figsize=(12,8))
sns.barplot(cnt.values, cnt.index, alpha=0.8, color='gray')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Product ID', fontsize=12)
plt.title("Top 10 products", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(dataPP['Value'], color='green')
plt.xlabel('Value', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.show()

### Point-of-sale

####  Quarterly analysis
###### January, February, and March (Q1)
###### April, May, and June (Q2)
###### July, August, and September (Q3)
###### October, November, and December (Q4)

In [ ]:
fig = px.line(dataPP.sort_values(by='Date'), x="Date", y="Value")
fig.show()

In [ ]:
fig = px.bar(data_values, x="total_bill", y="day", orientation='h')
fig.show()

#### Association rules

In [4]:
#USAR 
col_list = ["ProductFamily_ID", "Point-of-Sale_ID","ProductName_ID","Units","Date","Quarter"]
dataPP =  pd.read_csv(pathPP, usecols=col_list, nrows=1000000,dtype={'ProductFamily_ID': str, 'Point-of-Sale_ID': str, 'ProductName_ID': str})

In [5]:
dataPP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   ProductFamily_ID  1000000 non-null  object 
 1   ProductName_ID    1000000 non-null  object 
 2   Point-of-Sale_ID  1000000 non-null  object 
 3   Date              1000000 non-null  object 
 4   Units             1000000 non-null  float64
dtypes: float64(1), object(4)
memory usage: 38.1+ MB


In [7]:
dataPP['Date']= pd.to_datetime(dataPP['Date'])
dataPP['year'] = pd.DatetimeIndex(dataPP['Date']).year

In [8]:
dataPP['Date'] = dataPP['Date'].astype(str)

dataPP['TID'] = dataPP['ProductFamily_ID'] + dataPP['Point-of-Sale_ID'] + ' ' + dataPP['Date']

In [ ]:
# Selecionar Year / Date / PoS
fam =  dataPP[(dataPP['Point-of-Sale_ID'] == '1') & (dataPP['Quarter'] == 1) & (dataPP['Year'] == '2016')]

In [ ]:
fam = fam[['ProductName_ID','TID']]

In [ ]:
fam_pivot = pd.pivot_table(fam, index='TID', columns='ProductName_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)

In [ ]:
fam_pivot

In [ ]:
#fam.to_csv('fam_sample.csv.gz', compression='gzip')

In [ ]:
frequent_itemsets = apriori(fam_pivot, min_support=0.05, use_colnames=True)
frequent_itemsets.head()

In [ ]:
# Generate the association rules - by confidence
rulesConfidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.50)
rulesConfidence.sort_values(by='confidence', ascending=False, inplace=True)
rulesConfidence.head()

In [ ]:
# Generate the association rules - by lift
rulesLift = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rulesLift.sort_values(by='lift', ascending=False, inplace=True)
rulesLift.head()

In [ ]:
data_scatter = dict(type='scatter',
    y=rulesConfidence['confidence'][:10],
    x=rulesConfidence['support'][:10],
    #text=rulesLift.consequents,
    #mode='markers',
    #marker=dict(
    #size=rulesLift['support'],
    hovertemplate='Lift: ' + rulesLift["lift"].astype(str) + '<br>'+
                    'Confidence: ' + rulesLift['confidence'].astype(str) + '<br>'+
                    'Support: ' + rulesLift['support'].astype(str) + '<br>'+    
                    'Antecedents: ' + rulesLift['antecedents'].astype(str) + '<br>'+
                    'Consequents: ' + rulesLift['consequents'].astype(str) + '<br>'
    '<extra></extra>',
    mode='markers',
    marker=dict(size=8,
                color=rulesLift['lift'],
                colorscale='dense',
                showscale=True,
                line_width=2),
    )


fig = go.Figure(data=data_scatter)
fig.show()